<a href="https://colab.research.google.com/github/littleboss95/prueba/blob/master/Keyword_Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Activar Librerias
# Librerias para poder ejecutar el script

%%capture

!pip install tqdm

from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm
import json
import csv, sys
import pandas as pd
from google.colab import files
import requests
import shutil
import re

import ipywidgets as widgets
from IPython.display import display, HTML
from IPython.display import display
from ipywidgets import GridBox, Layout

from http.client import HTTPSConnection
from base64 import b64encode
from json import loads
from json import dumps
import warnings

import urllib
from string import ascii_lowercase


# Ignorar las advertencias específicas de pandas FutureWarning
warnings.filterwarnings('ignore', category=FutureWarning)

class RestClient:
    domain = "api.dataforseo.com"

    def __init__(self, username, password):
        self.username = username
        self.password = password

    def request(self, path, method, data=None):
        connection = HTTPSConnection(self.domain)
        try:
            base64_bytes = b64encode(
                ("%s:%s" % (self.username, self.password)).encode("ascii")
                ).decode("ascii")
            headers = {'Authorization' : 'Basic %s' %  base64_bytes, 'Content-Encoding' : 'gzip'}
            connection.request(method, path, headers=headers, body=data)
            response = connection.getresponse()
            return loads(response.read().decode())
        finally:
            connection.close()

    def get(self, path):
        return self.request(path, 'GET')

    def post(self, path, data):
        if isinstance(data, str):
            data_str = data
        else:
            data_str = dumps(data)
        return self.request(path, 'POST', data_str)

client = RestClient("jaime.sanchez@bigseo.com", "546754bd1dea034a")
#client = RestClient("adrian.garcia@bigseo.com", "7f673429c0b68694")
#client = RestClient("contact@bigseoagency.com", "2dc95d1f461f1045")
#client = RestClient("analytics@bigseoagency.com", "be18030100bbf265")


gmb=0

access_id = "mozscape-wfObEBqoM8"
secret_key = "uaqKDOqQw4198dJiVjtuMu5jysKbdwfw"

headers = {
    "x-moz-token": "QEbJW5snXJ9LINc7u0YCWtbcNlQewDY01UjeCfLVPi961ojU8efeTIdEfD3XasrM",
    "Content-Type": "application/json",
}

auth = (access_id, secret_key)
url = "https://lsapi.seomoz.com/v2/url_metrics"

In [ ]:
# @title Keyword Research

Pais = "Spain" # @param ["Spain","USA","Mexico","United Kingdom","India","Romania","Chile","Peru","Colombia"]
City = "Ninguna" # @param ["Barcelona","Madrid","Asturias","Ninguna","Los Angeles","New York","Murcia","Santa Cruz de Tenerife (Provincia)","Badajoz (provincia)","Washington","London"]
Keyword_Principal = "outsourcing" # @param {"type":"string"}
Keyword_2 = "servicio outsourcing" # @param {"type":"string"}
Keyword_3 = "agencia outsourcing" # @param {"type":"string"}
Keyword_4 = "empresa outsourcing" # @param {"type":"string"}
Keyword_5 = "outsourcing barcelona" # @param {"type":"string"}
Keyword_6 = "outsourcing administrativo" # @param {"type":"string"}
Keyword_7 = "outsourcing administrativo contable" # @param {"type":"string"}
Keyword_8 = "business process outsourcing" # @param {"type":"string"}
Keyword_9 = "empresa bpo" # @param {"type":"string"}
Keyword_10 = "" # @param {"type":"string"}
Keyword_11 = "" # @param {"type":"string"}
Keyword_12 = "" # @param {"type":"string"}
Keyword_13 = "" # @param {"type":"string"}
Keyword_14 = "" # @param {"type":"string"}
Keyword_15 = "" # @param {"type":"string"}
Keyword_16 = "" # @param {"type":"string"}
Keyword_17 = "" # @param {"type":"string"}
Keyword_18 = "" # @param {"type":"string"}
Keyword_19 = "" # @param {"type":"string"}
Keyword_20 = "" # @param {"type":"string"}

Keywords_Semilla = [
    Keyword_Principal, Keyword_2, Keyword_3, Keyword_4, Keyword_5,
    Keyword_6, Keyword_7, Keyword_8, Keyword_9, Keyword_10,
    Keyword_11, Keyword_12, Keyword_13, Keyword_14, Keyword_15,
    Keyword_16, Keyword_17, Keyword_18, Keyword_19, Keyword_20
]

Keywords_Semilla = [kw for kw in Keywords_Semilla if kw]


# Configuraciones de idioma y ubicación basadas en el país o ciudad seleccionados
config = {
    'Barcelona': {'code': 1005424, 'hl': 'es', 'Domain': 'google.es', 'country_code': 'es'},
    'Madrid': {'code': 1005493, 'hl': 'es', 'Domain': 'google.es', 'country_code': 'es'},
    'Murcia': {'code': 1005501, 'hl': 'es', 'Domain': 'google.es', 'country_code': 'es'},
    'Asturias': {'code': 20286, 'hl': 'es', 'Domain': 'google.es', 'country_code': 'es'},
    'Washington': {'code': 1027327, 'hl': 'en', 'Domain': 'google.com', 'country_code': 'us'},
    'Los Angeles': {'code': 200803, 'hl': 'en', 'Domain': 'google.com', 'country_code': 'us'},
    'New York': {'code': 200501, 'hl': 'en', 'Domain': 'google.com', 'country_code': 'us'},
    'USA': {'code': 2840, 'hl': 'en', 'Domain': 'google.com', 'country_code': 'us'},
    'Mexico': {'code': 2484, 'hl': 'es-419', 'Domain': 'google.com.mx', 'country_code': 'mx'},
    'Spain': {'code': 2724, 'hl': 'es', 'Domain': 'google.es', 'country_code': 'es'},
    #'Chile': {'code': 2152, 'hl': 'es-419'},
    #'Peru': {'code': 2604, 'hl': 'es-419'},
    #'Colombia': {'code': 2170, 'hl': 'es-419'},
    #'Romania': {'code': 2642, 'hl': 'ro'},
    'United Kingdom': {'code': 2826, 'hl': 'en', 'Domain': 'google.co.uk', 'country_code': 'uk'},
    'London': {'code': 20339, 'hl': 'en', 'Domain': 'google.co.uk', 'country_code': 'uk'},
    'Santa Cruz de Tenerife (Provincia)': {'code': 20295, 'hl': 'es', 'Domain': 'google.es', 'country_code': 'es'},
    'Badajoz (provincia)': {'code': 1005475, 'hl': 'es', 'Domain': 'google.es', 'country_code': 'es'},
    'India': {'code': 2356, 'hl': 'en', 'Domain': 'google.com', 'country_code': 'in'}
}
hl = config.get(City, config[Pais])['hl']
code = config.get(City, config[Pais])['code']
Domain = config.get(City, config[Pais])['Domain']
country_code = config.get(City, config[Pais])['country_code']

def get_results(query, Pais):
    query = urllib.parse.quote_plus(query)
    try:
        response = requests.get(f"https://suggestqueries.google.com/complete/search?output=chrome&google_domain={Domain}&gl={Pais}&hl={hl}&q={query}")

        if response.status_code == 200:  # Verifica que la respuesta sea OK
            results = json.loads(response.text)
            return results
        else:
            print(f"Error {response.status_code}: {response.text}")
            print(f"Keyword no válida: {query}")
            return None

    except requests.RequestException as e:
        print(f"RequestException: {e}")
        return None

def format_results(results):
    if results is None:  # Verifica que los resultados no sean None antes de proceder
        return []

    suggestions = []
    for i, value in enumerate(results[1]):
        suggestion = {'termino': value, 'relevance': results[4]['google:suggestrelevance'][i]}
        suggestions.append(suggestion)
    return suggestions

def get_expanded_terms(query):

    if hl in ['es', 'es-419']:

      expanded_term_prefixes = ['quien es *', 'que es *', 'donde esta *', 'cuando puede *', 'por que es *',
                              'el mejor *','la mejor *', 'barato *', 'el peor *', 'la peor *', 'es *', 'que *',
                              'cuando *', 'porque *', 'como *', 'quien *']

    if hl == 'en':

      expanded_term_prefixes = ['who is *', 'what is *', 'where is *', 'when can *', 'why is *',
                                  'the best *', 'the cheapest *', 'the worst *', 'is *', 'what *',
                                  'when *', 'why *', 'how *', 'who *']

    if hl == 'de':

      expanded_term_prefixes = ['wer ist *', 'was ist *', 'wo ist *', 'wann kann *', 'warum ist *',
                                  'das Beste *', 'das Günstigste *', 'das Schlechteste *', 'ist *', 'was *',
                                  'wann *', 'warum *', 'wie *', 'wer *']

    terms = []
    terms.append(query)

    for term in expanded_term_prefixes: #iteramos sobre los prefijos
        terms.append(term + ' ' + query)
        terms.append(term + ' ' + query + ' *')

    for term in ascii_lowercase: # iteramos sobre el alfabeto
        terms.append(query + ' ' + term)
        terms.append(query + ' * ' + term)
        terms.append(query + ' ' + term + ' *')

        # Concatenaciones con el término *antes* de la query
        terms.append(term + ' ' + query)

    return terms

def get_expanded_suggestions(query,Pais):

    all_results = []

    expanded_terms = get_expanded_terms(query)

    for term in expanded_terms:
        results = get_results(term,Pais)
        results = format_results(results)
        all_results = all_results + results
        all_results = sorted(all_results, key=lambda k: k['relevance'], reverse=True)
    return all_results

def google_autocomplete(query,country, include_expanded=True):

    results = get_expanded_suggestions(query,Pais)
    df = pd.DataFrame(results)
    return df

banned_words =[]

def filter_suggestions(dataframe, banned_words, current_keywords):
    banned_words = ["Valencia", "Barcelona", "Sevilla", "Zaragoza", "Malaga", "Murcia", "Mallorca", "Canaria",
    "Bilbao", "Alicante", "Cordoba", "Valladolid", "Vigo", "Gijon", "Hospitalet", "Vitoria",
    "Coruña", "Elche", "Granada", "Tarrasa", "Badalona", "Oviedo", "Cartagena", "Sabadell",
    "Jerez", "Tenerife", "Pamplona", "Almeria", "San Sebastian", "Leganes", "Getafe",
    "Burgos", "Albacete", "Alcorcon", "Alava", "Asturias", "Avila", "Badajoz", "Baleares",
    "Caceres", "Cadiz", "Cantabria", "Ciudad Real", "Cuenca", "canarias", "Girona", "Guadalajara",
    "Gipuzkoa", "Huelva", "Huesca", "Rioja", "Palmas", "Leon", "Lugo", "Madrid", "Navarra",
    "Salamanca", "Segovia", "Tarragona", "Teruel", "Toledo", "Vizcaya", "Ceuta", "Antequera",
    "Ibiza", "Menorca", "Formentera", "fuenlabrada", "chueca", "san sebastian de los reyes", "Palmanova", "uruguay", "chile", "colombia", "argentina", "mexico", "peru" "cap", "capitulo","foro", "foros", "forocoches"]

    #banned_words_localidad = ["uruguay", "chile", "colombia", "argentina", "mexico", "peru" "cap", "capitulo","foro", "foros", "forocoches", "gratis", "barato", "gratuito", "barata"]

    banned_words.extend(current_keywords)

    # Esta función elimina filas que contienen cualquier palabra prohibida
    return dataframe[~dataframe['termino'].str.contains('|'.join(banned_words), case=False, na=False)]


def get_non_brand_keywords(seed_keywords):
    """
    Si seed_keywords es un string, lo convertimos a lista de un solo elemento.
    Si es una lista de strings, simplemente la usamos.
    """
    if isinstance(seed_keywords, str):
        seed_keywords = [seed_keywords]

    # Ahora en vez de hacer post_data con una sola keyword,
    # lo hacemos con todas las que vengan en la lista seed_keywords.
    post_data = {
        0: {
            'location_code': code,
            'language_code': hl,
            'keywords': seed_keywords
        }
    }

    response = client.post(
        "/v3/keywords_data/google_ads/keywords_for_keywords/live",
        post_data
    )

    non_brand = []
    if response.get("status_code") == 20000:
        for item in response["tasks"][0].get("result", []):
            keyword_annotations = item.get("keyword_annotations")
            if not keyword_annotations:
                # Si es None o no es un diccionario, saltar
                continue

            # Sacamos 'concepts'
            concepts = keyword_annotations.get("concepts", [])
            if not concepts:
                # Si 'concepts' está vacío o es None, saltar
                continue

            for concept in concepts:
                if concept.get("name") == "Non-Brands":
                    non_brand.append(item["keyword"])
                    # Rompemos en cuanto encontramos 'Non-Brands'
                    break

    return non_brand


def main():
    # Paso 1: Obtener y expandir keywords
    initial_keywords = get_non_brand_keywords(Keywords_Semilla)
    expanded_keywords = list(initial_keywords)
    seen = set(expanded_keywords)

    new_kws = get_non_brand_keywords(initial_keywords[:20])
    for new_kw in new_kws:
        if new_kw not in seen:
            expanded_keywords.append(new_kw)
            seen.add(new_kw)

    df_ext = google_autocomplete(Keyword_Principal,country_code,True)

    df_ext = filter_suggestions(df_ext, banned_words, expanded_keywords)

    # Normalizar a minúsculas para eliminar duplicados insensibles a mayúsculas/minúsculas
    df_ext.loc[:, 'termino'] = df_ext['termino'].str.lower()

    # Eliminar duplicados
    df_ext = df_ext.drop_duplicates(subset=['termino'])

    # Convierte la columna 'termino' del dataframe a una lista
    expanded_long_keywords = df_ext['termino'].tolist()

    expanded_long_keywords.extend(expanded_keywords)

    # Resultados finales
    print("\nKeywords iniciales:", len(initial_keywords))
    print("\nKeywords expandidas:", len(expanded_keywords))
    print("\nKeywords expandidas + long tails:", len(expanded_long_keywords))

  # Crea un DataFrame a partir de la lista y especifica el nombre de la columna
    df_keywords = pd.DataFrame(expanded_long_keywords, columns=["Keywords"])

    # Exporta el DataFrame a un archivo Excel. (Para trabajar con .xlsx, asegúrate de tener instalado 'openpyxl' o 'XlsxWriter')
    df_keywords.to_excel("expanded_long_keywords.xlsx", index=False)
    files.download('expanded_long_keywords.xlsx')

    print("\nEl archivo 'expanded_long_keywords.xlsx' se ha generado correctamente.")

# Ejecutamos si es script principal
if __name__ == "__main__":
    main()


Keywords iniciales: 631

Keywords expandidas: 804

Keywords expandidas + long tails: 805


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


El archivo 'expanded_long_keywords.xlsx' se ha generado correctamente.
